In [43]:
import re

In [1]:
with open('data/input_16.txt') as fh:
    file_input = fh.read().strip()

In [4]:
data = file_input.split("\n")

In [26]:
tests = data[:3104]
program = data[3106:]

In [38]:
reg = [0,0,0,0]
ops = {
    'addr': lambda a, b: (reg[a] + reg[b]),
    'addi': lambda a, b: (reg[a] + b),
    'mulr': lambda a, b: (reg[a] * reg[b]),
    'muli': lambda a, b: (reg[a] * b),
    'banr': lambda a, b: (reg[a] & reg[b]),
    'bani': lambda a, b: (reg[a] & b),
    'borr': lambda a, b: (reg[a] | reg[b]),
    'bori': lambda a, b: (reg[a] | b),
    'setr': lambda a, b: (reg[a]),
    'seti': lambda a, b: (a),
    'gtir': lambda a, b: (1 if a > reg[b] else 0),
    'gtri': lambda a, b: (1 if reg[a] > b else 0),
    'gtrr': lambda a, b: (1 if reg[a] > reg[b] else 0),
    'eqir': lambda a, b: (1 if a == reg[b] else 0),
    'eqri': lambda a, b: (1 if reg[a] == b else 0),
    'eqrr': lambda a, b: (1 if reg[a] == reg[b] else 0),
}

In [37]:
test_input = """Before: [3, 2, 1, 1]
9 2 1 2
After:  [3, 2, 2, 1]"""

In [40]:
init = [3, 2, 1, 1]
cmp = [3, 2, 2, 1]
cand = []
for op in ops:
    reg = init[:]
    reg[2] = ops[op](2, 1)
    if reg == cmp:
        cand.append(op)
print (len(cand), cand)

(3, ['seti', 'mulr', 'addi'])


In [105]:
total = []
n = 0
for i in range(0, len(tests), 4):
    init = list(map(int, re.findall('\d+', tests[i])))
    args = list(map(int, re.findall('\d+', tests[i+1])))
    cmp = list(map(int, re.findall('\d+', tests[i+2])))
    cand = []
    for op in ops:
        reg = init[:]
        reg[args[3]] = ops[op](args[1], args[2])
        if reg == cmp:
            cand.append(op)
    total.append((args[0], set(cand)))
    if len(cand) >= 3:
        n += 1
print("num samples: {}".format(n))

num samples: 612


In [106]:
op_map = {i:set(ops.keys()) for i in range(16)}


for op, cand in total:
    op_map[op] = op_map[op] & cand

final_map = {}
while sum([len(op_map[op]) for op in op_map]):
    for op in op_map:
        if len(op_map[op]) == 1:
            final_map[op] = tuple(op_map[op])[0]
            op_map = {o: op_map[o] - op_map[op] for o in op_map }
    

In [107]:
final_map

{0: 'addi',
 1: 'bani',
 2: 'gtir',
 3: 'borr',
 4: 'eqrr',
 5: 'bori',
 6: 'gtrr',
 7: 'setr',
 8: 'muli',
 9: 'seti',
 10: 'banr',
 11: 'gtri',
 12: 'eqir',
 13: 'eqri',
 14: 'addr',
 15: 'mulr'}

In [110]:
reg = [0, 0, 0, 0]
for line in program:
    args = list(map(int, re.findall('\d+', line)))
    reg[args[3]] = ops[final_map[args[0]]](args[1], args[2])
print(reg)

[485, 3, 6, 485]


In [111]:
print("Part B: {}".format(reg[0]))

Part B: 485
